# **Render molecular structures with isosurface (support colormap) by POVRAY**

<hr style="height:1px;border:none;color:#cccccc;background-color:#cccccc;" />
<p style="text-align: justify;font-size:15px">  
    This is a tool to plot electrostatic mapping on charge density from VASP calculations.
    The users only need to upload CHGCAR and LOCPOT from VASP calculations. 
    User can choice the cutoff for the isosurface of the charge density. The transparency
    of the isosurface can be tuned by a slider. It will generate a plot as shown below
    (benenze molecule):
</p>

<div>
<img src="../images/electrostatic_colormap.png" alt="drawing" style="width:300px;"/>
<img src="../images/electrostatic_trans.png" alt="drawing" style="width:300px;"/>
</div>

In [ ]:
import nglview as nv
from ase.io import read, write
import numpy as np
from vapory import *
import json
from ase.calculators.vasp import VaspChargeDensity
from skimage import measure
from numpy.linalg import norm
from scipy.interpolate import RegularGridInterpolator
import copy
from matplotlib import cm
from ipywidgets import *
import matplotlib

style = {'description_width': 'initial'}

In [ ]:
with open("colors.json", "r") as fs:
    tcolors = json.load(fs)
    
with open("radius.json", "r") as fx:
    radius = json.load(fx)
    
fs.close()
fx.close()

color_theme = Dropdown(
    options=['jmol', 'xcrysden'],
    value='jmol',
    description='Color theme: ',
    disabled=False,
    style = style
)

cp = ColorPicker(
    concise=False,
    description='Background color',
    value='white',
    disabled=False,
    style = style
)

def background_color_change(b):
    view.background = cp.value

cp.observe(background_color_change, names="value")

def _prepare_payload(file_format=None):
    """Prepare binary information."""
    with open('nglview.png', 'rb') as raw:
        return base64.b64encode(raw.read()).decode()
    
    
def _download(payload, filename):
    """Download payload as a file named as filename."""
    javas = Javascript("""
        var link = document.createElement('a');
        link.href = "data:;base64,{payload}"
        link.download = "{filename}"
        document.body.appendChild(link);
        link.click();
        document.body.removeChild(link);
        """.format(payload=payload, filename=filename))
    display(javas)

In [ ]:
def vector_rotation(v):
    global zfactor
    A = view._camera_orientation
    A = np.array(A)
    A=A.reshape(4,4)
    A=np.transpose(A)

    zfactor = norm(A[0, 0:3])
    A[0:3, 0:3] = A[0:3, 0:3]/zfactor


    v = v + A[0:3, 3];
    w = A[0:3, 0:3].dot(v)
    return np.array([-w[0], w[1], w[2]])

In [ ]:
def _on_chg_upload(change):
    global C, view
    """When file upload button is pressed."""
    fchg.disabled = True
    for fname, item in change['new'].items():
        if fname == 'CHGCAR':
            with open("./CHGCAR", "wb") as fc:
                fc.write(item["content"])
            fc.close()
            C = VaspChargeDensity("CHGCAR")
            for comp_id in view._ngl_component_ids:
                view.remove_component(comp_id)
            view.add_component(nv.ASEStructure(C.atoms[-1]))
            view.add_unitcell()
            view.add_ball_and_stick(aspectRatio=4)
        lchg.value = "CHGCAR uploaded"
    fchg.disabled = False
    
def _on_pot_upload(change):
    global L
    """When file upload button is pressed."""
    fpot.disabled = True
    for fname, item in change['new'].items():
        if fname == 'LOCPOT':
            with open("./LOCPOT", "wb") as fp:
                fp.write(item["content"])
            fp.close()
            L = VaspChargeDensity("LOCPOT")
        lpot.value = "LOCPOT uploaded"
    fpot.disabled = False

        

fchg = FileUpload()
fpot = FileUpload()

fchg.observe(_on_chg_upload, names='value')
fpot.observe(_on_pot_upload, names='value')

lchg = Label(value="Please upload CHGCAR")
lpot = Label(value="Please upload LOCPOT")

cutoff = Textarea(value='0.02', description='Isosurface cutoff:', disabled=False)

In [ ]:
view = nv.NGLWidget(gui=False)
view.camera='perspective'
view.background='white'

In [ ]:
def pre_computing():
    global aa, verts, faces, normals, tlist, ifaces
    charge = C.chg[-1]
    cells = C.atoms[-1].get_cell()
    aa = C.atoms[-1]
    locpot = L.chg[-1]

    x = cells[0][0]/(np.shape(charge)[0] -1.0)
    y = cells[1][1]/(np.shape(charge)[1] -1.0)
    z = cells[2][2]/(np.shape(charge)[2] -1.0)

    #charge[:, :, 0:80] = 0.000
    verts, faces, normals, values = measure.marching_cubes(charge, float(cutoff.value), spacing = (x, y, z))

    origin_verts = copy.deepcopy(verts)

    for x, y in enumerate(verts):
        verts[x] = vector_rotation(y)

    x1 = np.linspace(0, cells[0][0], np.shape(charge)[0])
    y1 = np.linspace(0, cells[1][1], np.shape(charge)[1])
    z1 = np.linspace(0, cells[2][2], np.shape(charge)[2])

    interpolating_function = RegularGridInterpolator((x1, y1, z1), locpot)

    clist = [];
    for i in faces:
        clist.append(interpolating_function(origin_verts[i[0]]))
        clist.append(interpolating_function(origin_verts[i[1]]))
        clist.append(interpolating_function(origin_verts[i[2]]))


    clist = np.array(clist)

    clist = (clist-min(clist))/(max(clist)-min(clist))

    tlist = [];
    pcolor = cm.get_cmap('jet');

    for i in clist:
        k = pcolor(i)
        k = k[0]
        w = Texture(Pigment('rgbt', [k[0], k[1], k[2], transparency.value]))
                    #Finish('phong', 1, 'ambient', 0.1, 'diffuse', 0.9))
        tlist.append(w)

    ifaces = [];
    for x, y in enumerate(faces):
        ifaces.append(y.tolist())
        ifaces.append(3*x)
        ifaces.append(3*x+1)
        ifaces.append(3*x+2)

In [ ]:
bond_factor = FloatSlider(value = 1.2, min = 0.5, max = 2.0, description="Factor k: ", style = style)
transparency = FloatSlider(value = 0.0, min = 0.0, max = 1.0, description="Isosurface transparency: ", style = style)

bcell = Checkbox(
    value=True,
    description='Show cellbox',
    disabled=False,
    indent=False
)

def checkbox_change(b):
    global view
    if bcell.value:
        view.add_unitcell()
    else:
        view.remove_unitcell()

bcell.observe(checkbox_change, names='value')

In [ ]:
def render_images(b):
    br.disabled = True
    bd.disabled = True
    pre_computing()
    surfaces = Mesh2(VertexVectors(len(verts), *verts), NormalVectors(len(normals), *normals),
                     TextureList(len(tlist), *tlist),
                     FaceIndices(len(faces), *ifaces))

    spheres = [];

    colors = tcolors[color_theme.value]
    
    for i in aa:
        sphere = Sphere( vector_rotation([i.x, i.y, i.z]), radius[i.symbol], 
                        Texture(Pigment( 'color', np.array(colors[i.symbol]))), 
                            Finish('phong', 0.9,'reflection', 0.05))
        spheres.append(sphere)

    bonds = [];
    for x, i in enumerate(aa):
        for j in aa[x+1:]:
            v1 = vector_rotation(np.array([i.x, i.y, i.z]))
            v2 = vector_rotation(np.array([j.x, j.y, j.z]))

            if i.symbol == 'H' and j.symbol == 'H':
                continue;

            if norm(v1-v2) < bond_factor.value*(radius[i.symbol] + radius[j.symbol]):
                midi = v1 + (v2-v1)*radius[i.symbol]/(radius[i.symbol] + radius[j.symbol]);
                bond = Cylinder(v1, midi, 0.2, Pigment('color', np.array(colors[i.symbol])),
                                Finish('phong', 0.8,'reflection', 0.05))
                bonds.append(bond)
                bond = Cylinder(v2, midi, 0.2, Pigment('color', np.array(colors[j.symbol])),
                                Finish('phong', 0.8,'reflection', 0.05))
                bonds.append(bond)

    vertices = [];
    
    vx = np.array(aa.get_cell()[0]);
    vy = np.array(aa.get_cell()[1]);
    vz = np.array(aa.get_cell()[2]);

    vertices.append(vector_rotation(np.array([0, 0, 0])));
    vertices.append(vector_rotation(vx));
    vertices.append(vector_rotation(vy));
    vertices.append(vector_rotation(vz));
    
    vertices.append(vector_rotation(vx+vy));
    vertices.append(vector_rotation(vx+vz));
    vertices.append(vector_rotation(vy+vz));
    vertices.append(vector_rotation(vx+vy+vz));

    edges = [];
    for x, i in enumerate(vertices):
        for y, j in enumerate(vertices):
            if y > x:
                if norm(np.cross(i-j, vertices[1]-vertices[0])) < 0.001 or norm(np.cross(i-j, vertices[2]-vertices[0])) < 0.001 or norm(np.cross(i-j, vertices[3]-vertices[0])) < 0.001:
                    edge = Cylinder(i, j, 0.06, Texture(Pigment( 'color', [212/255.0,175/255.0,55/255.0])), 
                                    Finish('phong', 0.9,'reflection', 0.01))
                    edges.append(edge)
    
    
    camera = Camera('location', [0, 0, -zfactor/1.5], 'look_at', [0.0, 0.0, 0.0])
    light1 = LightSource([0, 0, -100.0], 'color',  [1.5, 1.5, 1.5])

    background = Background( "color", np.array(matplotlib.colors.to_rgb(view.background)))
    
    objects = [light1, background] + [surfaces] + spheres + bonds
    if bcell.value:
        objects += edges

    scene = Scene( camera, objects= objects)
    scene.render("nglview.png", width=5000, height=5000, remove_temp=False)
    br.disabled = False
    bd.disabled = False
    

def download_images(b):
    _download(payload=_prepare_payload(), filename="nglview.png")    


br = Button(description = 'Render')
bd = Button(description = 'Download', disabled = True)

br.on_click(render_images)
bd.on_click(download_images)

In [ ]:
display(HBox([fchg, lchg]))
display(HBox([fpot, lpot]))

In [ ]:
display(bond_factor)

In [ ]:
display(cp, bcell, transparency)

In [ ]:
display(cutoff)

In [ ]:
display(view)

In [ ]:
display(HBox([br, bd]))